In [1]:
val sc = spark.sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
28,application_1606840862501_0008,spark,idle,Link,Link


SparkSession available as 'spark'.
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@5e9ce7f4


In [2]:
import org.apache.spark.sql.types.{FloatType,LongType}
import org.apache.spark.sql.types.{ArrayType, StructField, StructType}
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.types.{FloatType, LongType}
import org.apache.spark.sql.types.{ArrayType, StructField, StructType}
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql.expressions.Window


In [3]:
val df_all = spark.read.parquet("hdfs:///Projects/AMLSim/gan_sim/ts_sim_ano.parquet")

df_all: org.apache.spark.sql.DataFrame = [_1: double, _2: double ... 364 more fields]


In [4]:
df_all.count()

res1: Long = 11000


In [5]:
df_all.where($"target" === 1).count()

res2: Long = 1000


In [6]:
df_all.where($"target" === 0).count()

res3: Long = 10000


In [7]:
df_all.printSchema

root
 |-- _1: double (nullable = true)
 |-- _2: double (nullable = true)
 |-- _3: double (nullable = true)
 |-- _4: double (nullable = true)
 |-- _5: double (nullable = true)
 |-- _6: double (nullable = true)
 |-- _7: double (nullable = true)
 |-- _8: double (nullable = true)
 |-- _9: double (nullable = true)
 |-- _10: double (nullable = true)
 |-- _11: double (nullable = true)
 |-- _12: double (nullable = true)
 |-- _13: double (nullable = true)
 |-- _14: double (nullable = true)
 |-- _15: double (nullable = true)
 |-- _16: double (nullable = true)
 |-- _17: double (nullable = true)
 |-- _18: double (nullable = true)
 |-- _19: double (nullable = true)
 |-- _20: double (nullable = true)
 |-- _21: double (nullable = true)
 |-- _22: double (nullable = true)
 |-- _23: double (nullable = true)
 |-- _24: double (nullable = true)
 |-- _25: double (nullable = true)
 |-- _26: double (nullable = true)
 |-- _27: double (nullable = true)
 |-- _28: double (nullable = true)
 |-- _29: double (nullab

In [8]:
print(df_all.columns.toSeq)

WrappedArray(_1, _2, _3, _4, _5, _6, _7, _8, _9, _10, _11, _12, _13, _14, _15, _16, _17, _18, _19, _20, _21, _22, _23, _24, _25, _26, _27, _28, _29, _30, _31, _32, _33, _34, _35, _36, _37, _38, _39, _40, _41, _42, _43, _44, _45, _46, _47, _48, _49, _50, _51, _52, _53, _54, _55, _56, _57, _58, _59, _60, _61, _62, _63, _64, _65, _66, _67, _68, _69, _70, _71, _72, _73, _74, _75, _76, _77, _78, _79, _80, _81, _82, _83, _84, _85, _86, _87, _88, _89, _90, _91, _92, _93, _94, _95, _96, _97, _98, _99, _100, _101, _102, _103, _104, _105, _106, _107, _108, _109, _110, _111, _112, _113, _114, _115, _116, _117, _118, _119, _120, _121, _122, _123, _124, _125, _126, _127, _128, _129, _130, _131, _132, _133, _134, _135, _136, _137, _138, _139, _140, _141, _142, _143, _144, _145, _146, _147, _148, _149, _150, _151, _152, _153, _154, _155, _156, _157, _158, _159, _160, _161, _162, _163, _164, _165, _166, _167, _168, _169, _170, _171, _172, _173, _174, _175, _176, _177, _178, _179, _180, _181, _182, _18

In [9]:
print(df_all.columns.slice(0,365).map(a => a.toString ).toSeq)

WrappedArray(_1, _2, _3, _4, _5, _6, _7, _8, _9, _10, _11, _12, _13, _14, _15, _16, _17, _18, _19, _20, _21, _22, _23, _24, _25, _26, _27, _28, _29, _30, _31, _32, _33, _34, _35, _36, _37, _38, _39, _40, _41, _42, _43, _44, _45, _46, _47, _48, _49, _50, _51, _52, _53, _54, _55, _56, _57, _58, _59, _60, _61, _62, _63, _64, _65, _66, _67, _68, _69, _70, _71, _72, _73, _74, _75, _76, _77, _78, _79, _80, _81, _82, _83, _84, _85, _86, _87, _88, _89, _90, _91, _92, _93, _94, _95, _96, _97, _98, _99, _100, _101, _102, _103, _104, _105, _106, _107, _108, _109, _110, _111, _112, _113, _114, _115, _116, _117, _118, _119, _120, _121, _122, _123, _124, _125, _126, _127, _128, _129, _130, _131, _132, _133, _134, _135, _136, _137, _138, _139, _140, _141, _142, _143, _144, _145, _146, _147, _148, _149, _150, _151, _152, _153, _154, _155, _156, _157, _158, _159, _160, _161, _162, _163, _164, _165, _166, _167, _168, _169, _170, _171, _172, _173, _174, _175, _176, _177, _178, _179, _180, _181, _182, _18

In [10]:
import org.apache.spark.ml.feature.{MinMaxScaler,StandardScaler,Normalizer, MaxAbsScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().setInputCols(df_all.columns.slice(0,365).map(a => a.toString).toArray).setOutputCol("features")

val df_all_features = assembler.transform(df_all)

// scaler instance with the min(-1) and max(1) 
val mms = new MinMaxScaler().
    setInputCol("features").
    setOutputCol("mms").
    setMax(1).
    setMin(-1)

// scaler instance with the min(0) and max(1) 
val mms01 = new MinMaxScaler().
    setInputCol("features").
    setOutputCol("mms").
    setMax(1).
    setMin(0)

val stds = new StandardScaler().
    setInputCol("features").
    setOutputCol("stds")

val maxabsscaler = new MaxAbsScaler().
    setInputCol("features").
    setOutputCol("maxabsscaler")

val mms_tr = mms.setInputCol("features").setOutputCol("mms")
val mms01_tr = mms01.setInputCol("features").setOutputCol("mms01")
val mms_abs_tr = maxabsscaler.setInputCol("features").setOutputCol("maxabsscaler")
val stds_tr = stds.setInputCol("features").setOutputCol("stds")

val trainingFeaturesPipeline = (new Pipeline()
  .setStages(Array(mms_tr,mms01_tr,mms_abs_tr,stds_tr)))
val trainingFeaturesDF = trainingFeaturesPipeline.fit(df_all_features).transform(df_all_features)

import org.apache.spark.ml.feature.{MinMaxScaler, StandardScaler, Normalizer, MaxAbsScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.VectorAssembler
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_40917cf62260
df_all_features: org.apache.spark.sql.DataFrame = [_1: double, _2: double ... 365 more fields]
mms: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_60248418cf44
mms01: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_3fca657b3046
stds: org.apache.spark.ml.feature.StandardScaler = stdScal_f4e38dde8cb5
maxabsscaler: org.apache.spark.ml.feature.MaxAbsScaler = maxAbsScal_1d77ed1cb6c5
mms_tr: mms.type = minMaxScal_60248418cf44
mms01_tr: mms01.type = minMaxScal_3fca657b3046
mms_abs_tr: maxabsscaler.type = maxAbsScal_1d77ed1cb6c5
stds_tr: stds.type = stdScal_f4e38dde8cb5
trainingFeaturesPipeline: org.apache.spark.ml.Pipeline = pipeline_fc0b4e4ca527
trainingFeaturesDF: org.apache.spark.sql.DataFrame = [_1: double, _2: double 

In [11]:
trainingFeaturesDF.printSchema

root
 |-- _1: double (nullable = true)
 |-- _2: double (nullable = true)
 |-- _3: double (nullable = true)
 |-- _4: double (nullable = true)
 |-- _5: double (nullable = true)
 |-- _6: double (nullable = true)
 |-- _7: double (nullable = true)
 |-- _8: double (nullable = true)
 |-- _9: double (nullable = true)
 |-- _10: double (nullable = true)
 |-- _11: double (nullable = true)
 |-- _12: double (nullable = true)
 |-- _13: double (nullable = true)
 |-- _14: double (nullable = true)
 |-- _15: double (nullable = true)
 |-- _16: double (nullable = true)
 |-- _17: double (nullable = true)
 |-- _18: double (nullable = true)
 |-- _19: double (nullable = true)
 |-- _20: double (nullable = true)
 |-- _21: double (nullable = true)
 |-- _22: double (nullable = true)
 |-- _23: double (nullable = true)
 |-- _24: double (nullable = true)
 |-- _25: double (nullable = true)
 |-- _26: double (nullable = true)
 |-- _27: double (nullable = true)
 |-- _28: double (nullable = true)
 |-- _29: double (nullab

In [12]:
import org.apache.spark.ml.linalg.DenseVector
val toArr: Any => Array[Double] = _.asInstanceOf[DenseVector].toArray
val toArrUdf = udf(toArr)
val final_df_all = trainingFeaturesDF.select($"target",$"features",$"mms",$"mms01",$"maxabsscaler",$"stds").  
             withColumn("features",toArrUdf($"features").cast(ArrayType(FloatType))).
             withColumn("mms",toArrUdf($"mms").cast(ArrayType(FloatType))).
             withColumn("mms01",toArrUdf($"mms01").cast(ArrayType(FloatType))).
             withColumn("maxabsscaler",toArrUdf($"maxabsscaler").cast(ArrayType(FloatType))).
             withColumn("stds",toArrUdf($"stds").cast(ArrayType(FloatType)))

import org.apache.spark.ml.linalg.DenseVector
toArr: Any => Array[Double] = <function1>
toArrUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)
final_df_all: org.apache.spark.sql.DataFrame = [target: int, features: array<float> ... 4 more fields]


In [13]:
val van = final_df_all.where($"target"===1)
val ben = final_df_all.where($"target"===0) 

van: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: int, features: array<float> ... 4 more fields]
ben: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: int, features: array<float> ... 4 more fields]


In [14]:
// Now that the data has been prepared, let's split the dataset into a training and test dataframe
val Array(benTrainDF, benEvalDF) = final_df_all.randomSplit(Array(0.8, 0.02),seed = 12345)

benTrainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: int, features: array<float> ... 4 more fields]
benEvalDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: int, features: array<float> ... 4 more fields]


In [15]:
benTrainDF.count()

res19: Long = 10728


In [16]:
benEvalDF.count()

res20: Long = 272


In [17]:
val EvalDF = benEvalDF.union(van)

EvalDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: int, features: array<float> ... 4 more fields]


In [18]:
EvalDF.where($"target"===1).count()

res21: Long = 1023


In [19]:
EvalDF.where($"target"===0).count()

res22: Long = 249


In [20]:
benTrainDF.write.format("tfrecords").mode("overwrite").save("hdfs:///Projects/AMLSim/gan_sim/train.tfrecord")
EvalDF.orderBy(rand()).write.format("tfrecords").mode("overwrite").save("hdfs:///Projects/AMLSim/gan_sim/eval.tfrecord")

In [21]:
EvalDF.show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|target|            features|                 mms|               mms01|        maxabsscaler|                stds|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     0|[-2.8352332, -2.3...|[0.4119701, 0.576...|[0.70598507, 0.78...|[-0.09902263, -0....|[-2.5037782, -1.5...|
|     0|[-2.1789706, -2.2...|[0.44788998, 0.58...|[0.72394496, 0.79...|[-0.07610217, -0....|[-1.9242364, -1.4...|
|     0|[-1.9913867, -1.8...|[0.4581572, 0.608...|[0.7290786, 0.804...|[-0.069550656, -0...|[-1.758582, -1.22...|
|     0|[-1.6628408, -0.6...|[0.47613984, 0.68...|[0.7380699, 0.840...|[-0.05807595, -0....|[-1.4684452, -0.4...|
|     0|[-1.2729217, -2.2...|[0.49748167, 0.58...|[0.74874085, 0.79...|[-0.044457734, -0...|[-1.1241097, -1.4...|
|     0|[-1.2677194, -1.8...|[0.4977664, 0.607...|[0.7488832, 0.803...|[-0.04427604, -0.